# Current

## Create test example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import string, random, toolz, pandas, importlib, sys, os

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    193091.000000
mean        257.945269
std         256.685709
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3319.000000
dtype: float64

## Run current version

In [5]:
!git checkout main
#importlib.reload(dimod)

Switched to branch 'main'
Your branch and 'upstream/main' have diverged,
and have 1 and 2 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)


In [6]:
from dimod import BinaryPolynomial as BPold

In [7]:
BPold.__init__??

Signature: BPold.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        self._terms = terms = {}
        for term, bias in poly:

            fsterm = asfrozenset(term)

            # when SPIN-valued, s^2 == 1, so we need to handle that case
            # in BINARY, x^2 == x
            if len(fsterm) < len(term) and vartype is Vartype.SPIN:
                new = set()
                term = tuple(term)  # make sure it has .count
                for v in fsterm:
                    if term.count(v) % 2:
                        new.add(v)
                fsterm = frozenset(new)

            if fsterm in terms:
                terms[fsterm] += bias
            else:
                terms[fsterm] = bias

        self.vartype = 

In [8]:
%%prun -s cumtime -l 20
bp = BPold(terms, 'SPIN')

         9739394 function calls (9739314 primitive calls) in 28.259 seconds

   Ordered by: cumulative time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   28.259   28.259 {built-in method builtins.exec}
        1    0.000    0.000   28.259   28.259 <string>:1(<module>)
        1    0.000    0.000   28.259   28.259 decorators.py:289(new_f)
        1    2.699    2.699   28.259   28.259 polynomial.py:89(__init__)
  5492601   24.575    0.000   24.575    0.000 {method 'count' of 'tuple' objects}
   193091    0.614    0.000    0.645    0.000 polynomial.py:32(asfrozenset)
  3474226    0.298    0.000    0.298    0.000 {method 'add' of 'set' objects}
   386184    0.042    0.000    0.042    0.000 {built-in method builtins.len}
   193101    0.030    0.000    0.030    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
    

In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,182233.000000,182233.000000
mean,18.934386,1.059583
std,7.545862,9.112464
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,40.000000,3851.000000


# Exit interpreter
I just can't get module reloading to work properly here ...

In [10]:
## 
exit()
#del BPold

# Proposed

## Create test example

In [1]:
import string, random, toolz, pandas, importlib, sys, os

In [2]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [3]:
pandas.Series(len(term) for term, _ in terms).describe()

count    192689.000000
mean        258.485497
std         257.605560
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        2809.000000
dtype: float64

## Run proposed version

In [4]:
!git checkout binary-polynomial
!touch -c ./dimod/higherorder/polynomial.py
#importlib.reload(dimod)

Already on 'binary-polynomial'
Your branch is ahead of 'origin/binary-polynomial' by 1 commit.
  (use "git push" to publish your local commits)


In [5]:
from dimod import BinaryPolynomial as BPnew

In [6]:
BPnew.__init__??

Signature: BPnew.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms

        self._terms = terms = defaultdict(int)
        for term, bias in poly:
            terms[freeze_term(term, vartype)] += bias
        
        self.vartype = vartype
File:      ~/dimod/dimod/higherorder/polynomial.py
Type:      function


In [7]:
%%prun -s cumtime -l 20 
bp = BPnew(terms, 'SPIN')

         578270 function calls (578190 primitive calls) in 4.526 seconds

   Ordered by: cumulative time
   List reduced from 45 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.526    4.526 {built-in method builtins.exec}
        1    0.000    0.000    4.526    4.526 <string>:1(<module>)
        1    0.000    0.000    4.526    4.526 decorators.py:289(new_f)
        1    0.189    0.189    4.526    4.526 polynomial.py:111(__init__)
   192689    4.308    0.000    4.337    0.000 polynomial.py:43(freeze_term)
   192699    0.016    0.000    0.016    0.000 {built-in method builtins.isinstance}
   192690    0.013    0.000    0.013    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}
     41/1    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__

In [8]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,181909.000000,181909.000000
mean,18.914314,1.059260
std,7.543998,9.110785
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,40.000000,3848.000000
